In [ ]:
import requests
import pandas as pd
import time
import json

In [ ]:
def extract_match_details(match_detail):
    draft_timings_data = []
    draft_timings = match_detail['draft_timings']
    clean_match_detail = {}
    match_id = match_detail['match_id']
    for timing in draft_timings:
        timing['match_id'] = match_id
        draft_timings_data.append(timing)
        
    objectives_data = []
    
    objectives = match_detail['objectives']
    for objective in objectives:
        objective['match_id'] = match_id
        objectives_data.append(objective)
    
    player_stats_timedata_df = pd.DataFrame()
    player_item_time_df = pd.DataFrame()
    clean_players_df = pd.DataFrame()
    for player in match_detail['players']:
        try:
                player_stat = pd.DataFrame()
                player_stat['time'] = player['times']
                player_stat['gold'] = player['gold_t']
                player_stat['lh'] = player['lh_t']
                player_stat['dn'] = player['dn_t']
                player_stat['xp'] = player['xp_t']
                player_stat['player_id'] = [player['account_id']] * len(player['times'])
                player_stat['player_id'] = [match_id] * len(player['times'])
                
                player_stats_timedata_df = pd.concat([player_stats_timedata_df, player_stat])
                
                item_time = []
                for key, value in player['purchase_time'].items():
                    item_time.append(
                        {
                            "match_id": match_id,
                            "player_id": player['account_id'],
                            'item': key,
                            'time': value
                        }
                    )
                
                player_item_time_df = pd.concat([player_item_time_df, pd.DataFrame(item_time)])
                
                cp = {
                    'match_id': match_id
                }
                for key, value in player.items():
                    if type(value) in [dict, list]:
                        continue
                    cp[key] = value
                clean_players_df = pd.concat([clean_players_df, pd.DataFrame([cp])])
        except Exception:
                print('Error in match_id ', match_id)
                pass
        
    
    match_stats_df = pd.DataFrame()
    
    match_stats_df['time'] = range(len(match_detail['radiant_gold_adv']))
    match_stats_df['radiant_gold_adv'] = match_detail['radiant_gold_adv']
    match_stats_df['radiant_xp_adv'] = match_detail['radiant_xp_adv']
    match_stats_df['match_id'] = match_id * len(match_detail['radiant_gold_adv'])
    
    for key, value in match_detail.items():
        if type(value) in [dict, list]:
            continue
        clean_match_detail[key] = value
    
    return pd.DataFrame(draft_timings_data), pd.DataFrame(objectives_data), match_stats_df, clean_players_df, player_item_time_df, player_stats_timedata_df, pd.DataFrame([clean_match_detail])

In [ ]:
driver = "org.postgresql.Driver"

database_host = "*******"
database_port = "****" # update if you use a non-default port
database_name = "postgres"
table = "match_details"
user = "******"
password = "*******"

url = f"jdbc:postgresql://{database_host}:{database_port}/{database_name}"

remote_table = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

In [ ]:
remote_table.show()

+-------+----------+--------+----------+--------+---------+-----------+-------+-----------+-----------+-----------------+-------------+--------------------+-----------------+-----------------------+--------------------+----------------+----------+-------------+---------+-----+------+-------------+----------+---------------+----------------+-------------------+---------------------+------------+-----------------+--------------------+------------------+---------------+-------------+--------+-----+------+------+-------+--------------------+------------+---------+--------+-------+----------+
|version|  match_id|leagueid|start_time|duration|series_id|series_type|cluster|replay_salt|radiant_win|pre_game_duration|match_seq_num|tower_status_radiant|tower_status_dire|barracks_status_radiant|barracks_status_dire|first_blood_time|lobby_type|human_players|game_mode|flags|engine|radiant_score|dire_score|radiant_team_id|    radiant_name|       radiant_logo|radiant_team_complete|dire_team_id|        

In [ ]:
top_match_id = int(remote_table.describe("match_id").filter("summary = 'max'").select("match_id").first().asDict()['match_id'])

In [ ]:
def get_pro_macthes(match_id)
endpoint = 'https://api.opendota.com/api/proMatches?'

response = requests.get(endpoint)
if response.status_code == 200:
    pro_matches = response.json()
    print(pro_matches)
else:
    print('Failed to retrieve pro matches:', response.status_code)

[{'match_id': 7699538252, 'duration': 2674, 'start_time': 1713803251, 'radiant_team_id': 9381131, 'radiant_name': 'Uzumaki', 'dire_team_id': 8829924, 'dire_name': 'Rabbits of Revenge', 'leagueid': 15901, 'league_name': 'Ultras Dota Pro League  2023-24', 'series_id': None, 'series_type': None, 'radiant_score': 48, 'dire_score': 39, 'radiant_win': True, 'version': None}, {'match_id': 7699534164, 'duration': 1141, 'start_time': 1713803119, 'radiant_team_id': 9360721, 'radiant_name': 'KIBA ARMS', 'dire_team_id': 8831040, 'dire_name': 'Ghost Sheep', 'leagueid': 15901, 'league_name': 'Ultras Dota Pro League  2023-24', 'series_id': None, 'series_type': None, 'radiant_score': 40, 'dire_score': 7, 'radiant_win': True, 'version': None}, {'match_id': 7699443041, 'duration': 1378, 'start_time': 1713800224, 'radiant_team_id': 8831040, 'radiant_name': 'Ghost Sheep', 'dire_team_id': 9360721, 'dire_name': 'KIBA ARMS', 'leagueid': 15901, 'league_name': 'Ultras Dota Pro League  2023-24', 'series_id': No

In [ ]:
promatches_df = pd.DataFrame()
match_id = 9999999999999
while True:
    endpoint = f'https://api.opendota.com/api/proMatches?less_than_match_id={match_id}'
    
    response = requests.get(endpoint)
    if response.status_code == 200:
        pro_matches_json = response.json()
        print(len(pro_matches_json))
        if len(pro_matches_json) == 0:
            break
        data_df = pd.DataFrame(pro_matches_json)
        promatches_df = promatches_df.append(data_df)
    else:
        print('Failed to retrieve pro matches:', response.status_code)
        break
    
    
    print('Data collected = ', len(promatches_df))
    if len(promatches_df) > 500 or promatches_df['match_id'].min() < top_match_id:
        break

    match_id = promatches_df['match_id'].min()

100
Data collected =  100
100


<command-436461412861965>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  promatches_df = promatches_df.append(data_df)


Data collected =  200


<command-436461412861965>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  promatches_df = promatches_df.append(data_df)


In [ ]:
promatches_df['match_id'].min()

Out[22]: 7693019017

In [ ]:
top_match_id

Out[23]: 7694225732

In [ ]:
filtered_pro_matches = [pro_match for index, pro_match in promatches_df.iterrows() if pro_match['match_id'] > top_match_id]

In [ ]:
def fetch_match_details(match_id):
    url = f"https://api.opendota.com/api/matches/{match_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch details for match_id {match_id} response code[{response.status_code}]")
        return None
    

In [ ]:
draft_timing_final_df = None
objectives_match_final_df = None
match_stats_final_df = None 
player_match_final_df = None
player_item_time_final_df = None
player_stats_time_final_df = None
match_details_final_df = None


In [ ]:
i = 1
for match_det in filtered_pro_matches:
    print('processing ====== ',i,'/',len(filtered_pro_matches), ' ', i/len(filtered_pro_matches) *100,'%')
    match_detail = fetch_match_details(match_det['match_id'])
    if match_detail:
        draft_timings_df, objectives_df, match_stats_df, clean_players_df, player_item_time_df, player_stats_time_data_df, clean_match_df = extract_match_details(match_detail)

        if draft_timing_final_df is not None:
            draft_timing_final_df = pd.concat([draft_timing_final_df, draft_timings_df])
        else:
            draft_timing_final_df = draft_timings_df

        if objectives_match_final_df is not None:
            objectives_match_final_df = pd.concat([objectives_match_final_df, objectives_df])
        else:
            objectives_match_final_df = objectives_df
        
        if match_stats_final_df is not None:
            match_stats_final_df = pd.concat([match_stats_final_df, match_stats_df])
        else:
            match_stats_final_df = match_stats_df
        
        if player_match_final_df is not None:
            player_match_final_df = pd.concat([player_match_final_df, clean_players_df])
        else:
            player_match_final_df = clean_players_df
        
        if player_item_time_final_df is not None:
            player_item_time_final_df = pd.concat([player_item_time_final_df, player_item_time_df])
        else:
            player_item_time_final_df = player_item_time_df
        
        if player_stats_time_final_df is not None:
            player_stats_time_final_df = pd.concat([player_stats_time_final_df, player_stats_time_data_df])
        else:
            player_stats_time_final_df = player_stats_time_data_df
        
        if match_details_final_df is not None:
            match_details_final_df = pd.concat([match_details_final_df, clean_match_df])
        else:
            match_details_final_df = clean_match_df
    i += 1
    time.sleep(0.5)

processing ======  1 / 168   0.5952380952380952 %
processing ======  2 / 168   1.1904761904761905 %
processing ======  3 / 168   1.7857142857142856 %
processing ======  4 / 168   2.380952380952381 %
processing ======  5 / 168   2.976190476190476 %
processing ======  6 / 168   3.571428571428571 %
processing ======  7 / 168   4.166666666666666 %
processing ======  8 / 168   4.761904761904762 %
processing ======  9 / 168   5.357142857142857 %
processing ======  10 / 168   5.952380952380952 %
processing ======  11 / 168   6.547619047619048 %
processing ======  12 / 168   7.142857142857142 %
processing ======  13 / 168   7.738095238095238 %
processing ======  14 / 168   8.333333333333332 %
processing ======  15 / 168   8.928571428571429 %
processing ======  16 / 168   9.523809523809524 %
processing ======  17 / 168   10.119047619047619 %
processing ======  18 / 168   10.714285714285714 %
processing ======  19 / 168   11.30952380952381 %
processing ======  20 / 168   11.904761904761903 %
pro

In [ ]:
draft_timing_spark = spark.createDataFrame(draft_timing_final_df)
objectives_match_spark = spark.createDataFrame(objectives_match_final_df)
match_stats_spark =  spark.createDataFrame(match_stats_final_df) 
player_match_spark = spark.createDataFrame(player_match_final_df)
player_item_time_spark = spark.createDataFrame(player_item_time_final_df)
player_stats_time_spark = spark.createDataFrame(player_stats_time_final_df)
match_details_spark = spark.createDataFrame(match_details_final_df)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:467: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Could not convert 'npc_dota_badguys_tower1_top' with type str: tried to convert to int64
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


In [ ]:
draft_timing_spark.show()

+-----+-----+-----------+-------+-----------+----------+----------------+----------+
|order| pick|active_team|hero_id|player_slot|extra_time|total_time_taken|  match_id|
+-----+-----+-----------+-------+-----------+----------+----------------+----------+
|    1|false|          3|    120|       null|       130|              16|7699602043|
|    2|false|          2|     66|       null|       130|              19|7699602043|
|    3|false|          2|    114|       null|       130|              12|7699602043|
|    4|false|          3|     91|       null|       130|              12|7699602043|
|    5|false|          2|     87|       null|       130|              19|7699602043|
|    6|false|          2|     65|       null|       130|              45|7699602043|
|    7|false|          3|     52|       null|       121|              30|7699602043|
|    8| true|          3|     78|        6.0|       105|              18|7699602043|
|    9| true|          2|     19|        3.0|       121|         

In [ ]:
match_details_spark.show()

+-------+----------+--------+----------+--------+---------+-----------+-------+-----------+-----------+-----------------+-------------+--------------------+-----------------+-----------------------+--------------------+----------------+----------+-------------+---------+-----+------+-------------+----------+---------------+------------------+-------------------+---------------------+------------+------------------+-------------------+------------------+---------------+------------+--------+-----+------+--------+-------+--------------------+------+-------+
|version|  match_id|leagueid|start_time|duration|series_id|series_type|cluster|replay_salt|radiant_win|pre_game_duration|match_seq_num|tower_status_radiant|tower_status_dire|barracks_status_radiant|barracks_status_dire|first_blood_time|lobby_type|human_players|game_mode|flags|engine|radiant_score|dire_score|radiant_team_id|      radiant_name|       radiant_logo|radiant_team_complete|dire_team_id|         dire_name|          dire_logo|

In [ ]:
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}
draft_timing_spark.write.jdbc(url=url, table='draft_timing' , mode='append', properties=jdbc_properties)
objectives_match_spark.write.jdbc(url=url, table='objectives_match' , mode='append', properties=jdbc_properties)
match_stats_spark.write.jdbc(url=url, table='match_stats' , mode='append', properties=jdbc_properties)
player_match_spark.write.jdbc(url=url, table='player_match' , mode='append', properties=jdbc_properties)
player_item_time_spark.write.jdbc(url=url, table='player_item_time' , mode='append', properties=jdbc_properties)
player_stats_time_spark.write.jdbc(url=url, table='player_stats_time' , mode='append', properties=jdbc_properties)